In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy.stats import zscore

In [2]:
df = pd.read_csv('train.csv', index_col=0)

In [5]:
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [6]:
df.shape

(13730, 166)

# Primeiro vamos realizar uma análise exploratória dos dados, para compreender algumas questões.
    - Como estão distribuídas as notas por estado? 
    - As notas de quem fez a prova de língua estrangeira em inglês é maior em relação à quem fez em espanhol? 
    - A escolaridade dos pais do candidato influeciam na nota do candidato?
    - Como estão distribuídas as notas dos candidatos em função da renda mensal familiar? 

In [ ]:
len(df.columns)

In [ ]:
def print_heatmap_null():
    plt.figure(figsize=(20, 10))
    sns.heatmap(df.isnull(), cbar=False)
    
print_heatmap_null()

In [ ]:
# Vamos dropar colunas que possuem mais de 50% de nulos
cols_nan_values = []
for col in df.columns:
    if df[col].isnull().sum() / len(df[col]) > 0.5:
        cols_nan_values.append(col)
df.drop(cols_nan_values, axis=1, inplace=True)

In [ ]:
print_heatmap_null()

In [ ]:
cols_object_type = []
cols_number_type = []
for col in df.columns:
    if df[col].dtypes == 'object':
        cols_object_type.append(col)
    else: 
        cols_number_type.append(col)

In [ ]:
print("Qtde colunas do tipo object: " + str(len(cols_object_type)) + "\nQtde colunas do tipo numerica: " + str(len(cols_number_type)))

In [ ]:
['NU_INSCRICAO', 'NO_MUNICIPIO_RESIDENCIA', 'TP_SEXO', 'NO_MUNICIPIO_NASCIMENTO', 'SG_UF_NASCIMENTO']
cols_object_type